# DGCNN PointCloud Classification Tutorial

> **Skill level:** _Beginner_

Kaolin makes 3D deep learning easy, by providing all the _hard_/_tricky_ parts of coding up a 3D DL application. To get a feel for how easy training classifiers on 3D data has become, here's a quick demo.

In this tutorial, we will train a _dgcnn_pointcloud_ classifier, in about five lines of code!

We will use the `ModelNet10` dataset for this tutorial. The remainder of this tutorial will be broken down into the following components.

- [Downloading ModelNet10](#downloading-modelnet10)
- [DataLoading](#dataloading)
- [Training the pointcloud classifier](#training-the-pointcloud-classifier)

## Downloading ModelNet10

Note that the ModelNet10 dataset is provided ONLY for the convenience of academic research. Should you choose to download it, you must adhere to the original terms and copyright notice of the dataset. For convenience, we reproduce the original copyright from the dataset creators.

```
**Copyright**

All CAD models are downloaded from the Internet and the original authors hold the copyright of the CAD models. The label of the data was obtained by us via Amazon Mechanical Turk service and it is provided freely. This dataset is provided for the convenience of academic research only.
```

The ModelNet10 (10-class subset) dataset is available on the [Princeton ModelNet page](https://modelnet.cs.princeton.edu/). On this page, navigate to the ModelNet10 download link to obtain the dataset. We assume that it is unzipped and extracted to a location `MODELNET_DIRECTORY`.

## Warm-up

Before all the fun-stuff begins, let us import all necessary functions from `kaolin` and `torch`. A bit more on what the following modules do will become clear as we progress.

In [1]:
import time
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

from kaolin.datasets import ModelNet
from kaolin.models.dgcnn import DGCNNClassifier
import kaolin.transforms as tfs

   No module named 'nuscenes'


## Dataloading

Kaolin provides convenience functions to load popular 3D datasets (of course, ModelNet10). Assuming you have [installed Kaolin](../../README.md#installation-and-usage), fire up your favourite python interpreter, and execute the following commands.

To start, we will define a few important parameters:

In [2]:
modelnet_path = '/scratch/pushkalkatara/ModelNet10'
categories = ['chair', 'sofa']
num_points = 1024
device = 'cuda'

The `model_path` variable will hold the path to the ModelNet10 dataset. We will use the `categories` variable to specify which classes we want to learn to classify. `num_points` is the number of points we will sample from the mesh when transforming it to a pointcloud.

In [3]:
def to_device(inp):
    inp.to(device)
    return inp

transform = tfs.Compose([
    to_device,
    tfs.TriangleMeshToPointCloud(num_samples=num_points),
    tfs.NormalizePointCloud()
])

This command defines a `transform` that first converts a mesh representation to a pointcloud and then _normalizes_ it to be centered at the origin, and have a standard deviation of 1. Much like images, 3D data such as pointclouds need to be normalized for better classification performance.

In [4]:
train_loader = DataLoader(
    ModelNet(modelnet_path, categories=categories, split='train', transform=transform),
    batch_size=32, shuffle=True, drop_last=True
)

Phew, that was slightly long! But here's what it does. It creates a `DataLoader` object for the `ModelNet10` dataset. In particular, we are interested in loading only the `chair` and `sofa` categories. The `split='train'` argument indicates that we're loading the 'train' split.

Similarly, the test dataset can be loaded up as follows.

In [5]:
val_loader = DataLoader(
    ModelNet(modelnet_path, categories=categories, split='test',transform=transform),
    batch_size=16, shuffle=True, drop_last=False
)

## Training the pointcloud classifier

Now that all of the data is ready, we can train our classifier using the `DGCNNClassifier` class provided by Kaolin. The following line of code will train and validate a _PointNet_ classifier, which is probably the simplest of pointcloud neural architectures.

In [ ]:
model = DGCNNClassifier(device=device)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

start_time = time.time()

epocs = 1

for e in range(epocs):
    print("Epoch: ", e)
    train_loss = 0
    train_accuracy = 0
    num_batches = 0
    
    model.train()
    
    for idx, (data, attributes) in enumerate(tqdm(train_loader)):
        category = attributes['category'].cuda()
        pred = model(data.cuda())
        loss = criterion(pred, category.view(-1))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        pred_label = torch.argmax(pred, dim=1)
        train_accuracy += torch.mean((pred_label == category.view(-1)).float()).detach().cpu().item()
        num_batches += 1
    print('Train loss:', train_loss / num_batches)
    print('Train accuracy:', train_accuracy / num_batches)
    
    val_loss = 0.
    val_accuracy = 0.
    num_batches = 0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (data, attributes) in enumerate(tqdm(val_loader)):
            category = attributes['category'].cuda()
            pred = model(data.cuda())
            loss = criterion(pred, category.view(-1))
            val_loss += loss.item()
            
            pred_label = torch.argmax(pred, dim=1)
            val_accuracy += torch.mean((pred_label == category.view(-1)).float()).cpu().item()
            num_batches += 1
        
    print('Val loss:', val_loss / num_batches)
    print('Val accuracy:', val_accuracy / num_batches)

end_time = time.time()
print('Training time: {}'.format(end_time - start_time))


        


  0%|          | 0/49 [00:00<?, ?it/s]

Epoch:  0


 88%|████████▊ | 43/49 [02:53<00:23,  3.99s/it]